<a href="https://colab.research.google.com/github/iamfernandareis/nivel-2-telos/blob/main/simulador_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


In [10]:
from pyspark.sql import SparkSession
from google.colab import files
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lower, upper, trim, avg
import pandas as pd


In [3]:
spark = SparkSession.builder.appName("Dados Notas Alunos").getOrCreate()

uploaded = files.upload()

Saving dados_notas_alunos_8825a393a1.csv to dados_notas_alunos_8825a393a1.csv


In [4]:
nome = "dados_notas_alunos_8825a393a1.csv"

df = spark.read.csv(nome, sep=';', inferSchema=True, header=True)

## 1) Visualização e Análise Inicial dos Dados
Como analista de dados, desejo tratar os dados enviados ao sistema para
padronizá-los e assim, ser possível fazer uma análise futura, entendendo a
correlação entre cada variável e tipo de dado.

Critérios de Aceitação:


⮚  **Carregar o dataset e mostrar as primeiras linhas e o esquema do
DataFrame.**




In [5]:
df.show()

+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
| Id|   G�nero|RendaFamiliar|     EducacaoPaterna|PeriodoDeAula|FezAulaParticular|EstadoCivilDosPais|PraticaEsportes|FilhoMaisVelho|NumIrmaos|TransporteEscolar|HorasDeEstudo|notaDeMatematica|notaDePortugues|notaDeCiencias|
+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
|  0| feminino|         NULL|   superior completo|     matutino|              nao|            casado|   regularmente|           sim|        3|           onibus|          < 5|              71|             71|            74|
|  1| feminino|      grupo C| superior incompleto|     matutino|             NULL|            casado|  event

In [6]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- G�nero: string (nullable = true)
 |-- RendaFamiliar: string (nullable = true)
 |-- EducacaoPaterna: string (nullable = true)
 |-- PeriodoDeAula: string (nullable = true)
 |-- FezAulaParticular: string (nullable = true)
 |-- EstadoCivilDosPais: string (nullable = true)
 |-- PraticaEsportes: string (nullable = true)
 |-- FilhoMaisVelho: string (nullable = true)
 |-- NumIrmaos: integer (nullable = true)
 |-- TransporteEscolar: string (nullable = true)
 |-- HorasDeEstudo: string (nullable = true)
 |-- notaDeMatematica: integer (nullable = true)
 |-- notaDePortugues: integer (nullable = true)
 |-- notaDeCiencias: integer (nullable = true)



⮚ **Realizar uma análise descritiva básica (média, mediana, desvios).**

In [7]:
df_pandas = df.toPandas()

In [8]:
media_matematica = df_pandas["notaDeMatematica"].mean()
print(f"Média das notas de Matemática: {media_matematica:.2f}")

Média das notas de Matemática: 66.56


In [11]:
# Calcular a média usando PySpark
media_matematica = df.select(F.avg("notaDeMatematica")).collect()[0][0]
print(f"Média das notas de Matemática (PySpark): {media_matematica:.2f}")

Média das notas de Matemática (PySpark): 66.56


In [12]:
mediana_matematica = df_pandas["notaDeMatematica"].median()
print(f"Mediana das notas de Matemática: {mediana_matematica}")

Mediana das notas de Matemática: 67.0


## 2) Limpeza e Preparação dos Dados
Como analista de dados, preciso limpar e preparar os dados, garantindo que
estejam prontos para análises mais complexas.


Critérios de Aceitação:


⮚ **Identificar e tratar valores nulos.**


In [18]:
df.show()

+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
| Id|   G�nero|RendaFamiliar|     EducacaoPaterna|PeriodoDeAula|FezAulaParticular|EstadoCivilDosPais|PraticaEsportes|FilhoMaisVelho|NumIrmaos|TransporteEscolar|HorasDeEstudo|notaDeMatematica|notaDePortugues|notaDeCiencias|
+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
|  0| feminino|         NULL|   superior completo|     matutino|              nao|            casado|   regularmente|           sim|        3|           onibus|          < 5|              71|             71|            74|
|  1| feminino|      grupo C| superior incompleto|     matutino|             NULL|            casado|  event

In [20]:
df2 = df.fillna({
    "RendaFamiliar": "Desconhecida",
    "EducacaoPaterna": "Desconhecido",
    "PeriodoDeAula": "Desconhecido",
    "FezAulaParticular": "nao",
    "EstadoCivilDosPais": "Desconhecido",
    "PraticaEsportes": 0,
    "FilhoMaisVelho": "Desconhecido",
    "NumIrmaos": 0,
    "TransporteEscolar": "Desconhecido",
    "HorasDeEstudo": "Não Informado"
})


In [19]:
df2.show()

+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
| Id|   G�nero|RendaFamiliar|     EducacaoPaterna|PeriodoDeAula|FezAulaParticular|EstadoCivilDosPais|PraticaEsportes|FilhoMaisVelho|NumIrmaos|TransporteEscolar|HorasDeEstudo|notaDeMatematica|notaDePortugues|notaDeCiencias|
+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
|  0| feminino| Desconhecida|   superior completo|     matutino|              nao|            casado|   regularmente|           sim|        3|           onibus|          < 5|              71|             71|            74|
|  1| feminino|      grupo C| superior incompleto|     matutino|              nao|            casado|  event

In [21]:
df3 = df2.dropna()


⮚ **Normalizar e formatar as colunas conforme necessário.**

In [22]:
df3 = df3.withColumn("EducacaoPaterna", lower(trim(col("EducacaoPaterna"))))
df3 = df3.withColumn("EstadoCivilDosPais", lower(trim(col("EstadoCivilDosPais"))))
df3 = df3.withColumn("PeriodoDeAula", lower(trim(col("PeriodoDeAula"))))
df3 = df3.withColumn("FezAulaParticular", upper(trim(col("FezAulaParticular"))))
df3 = df3.withColumn("FilhoMaisVelho", upper(trim(col("FilhoMaisVelho"))))
df3 = df3.withColumn("TransporteEscolar", lower(trim(col("TransporteEscolar"))))

In [23]:
df3.show()

+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
| Id|   G�nero|RendaFamiliar|     EducacaoPaterna|PeriodoDeAula|FezAulaParticular|EstadoCivilDosPais|PraticaEsportes|FilhoMaisVelho|NumIrmaos|TransporteEscolar|HorasDeEstudo|notaDeMatematica|notaDePortugues|notaDeCiencias|
+---+---------+-------------+--------------------+-------------+-----------------+------------------+---------------+--------------+---------+-----------------+-------------+----------------+---------------+--------------+
|  0| feminino| Desconhecida|   superior completo|     matutino|              NAO|            casado|   regularmente|           SIM|        3|           onibus|          < 5|              71|             71|            74|
|  1| feminino|      grupo C| superior incompleto|     matutino|              NAO|            casado|  event

3) Análise Avançada: Relação entre Renda Familiar e Notas
Como analista de dados, quero explorar como a renda familiar influencia o
desempenho dos alunos.

Critérios de Aceitação:

⮚ Agrupar os dados por renda familiar e calcular a média das notas.
⮚ Analisar se existe uma correlação visível, por exemplo: alunos com mais
horas de estudo, que fizeram aula particular, que se exercitam, tendem a
ter uma média de notas maiores? Faça uma análise que traga esses
resultados.

In [24]:
df_grouped = df2.groupBy("RendaFamiliar").agg(
    avg("notaDeMatematica").alias("media_nota_matematica"),
    avg("notaDePortugues").alias("media_nota_portugues"),
    avg("notaDeCiencias").alias("media_nota_ciencias")
)


In [25]:
df_grouped.show()

+-------------+---------------------+--------------------+-------------------+
|RendaFamiliar|media_nota_matematica|media_nota_portugues|media_nota_ciencias|
+-------------+---------------------+--------------------+-------------------+
| Desconhecida|    66.18586956521739|   68.91576086956522|  67.90054347826087|
|      grupo A|    62.99188823794502|   66.78774222622803|  65.25191527715187|
|      grupo D|    67.66640010662401|   70.38224710115954|  70.89084366253499|
|      grupo C|    64.69344333478072|   68.43823273990448|  66.99924012158054|
|      grupo E|    75.29893590695373|   74.25142291512002|  72.67706013363029|
|      grupo B|   63.490216271884655|   67.32046000686577|  65.89512530037761|
+-------------+---------------------+--------------------+-------------------+



In [26]:
from pyspark.sql.functions import when

# Converter FezAulaParticular para numérico: 1 para 'sim', 0 para 'não'
df3 = df3.withColumn("FezAulaParticular", when(col("FezAulaParticular") == "sim", 1).otherwise(0))

# Converter TransporteEscolar para numérico: 1 para 'sim', 0 para 'não'
df3 = df3.withColumn("TransporteEscolar", when(col("TransporteEscolar") == "sim", 1).otherwise(0))

# Converter PraticaEsportes para numérico: 1 para 'sim', 0 para 'não'
df3 = df3.withColumn("PraticaEsportes", when(col("PraticaEsportes") == "sim", 1).otherwise(0))

# Converter HorasDeEstudo para numérico (caso necessário)
df3 = df3.withColumn("HorasDeEstudo", col("HorasDeEstudo").cast("double"))

# Verificar se todas as colunas necessárias estão no formato numérico
df3.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- G�nero: string (nullable = true)
 |-- RendaFamiliar: string (nullable = false)
 |-- EducacaoPaterna: string (nullable = false)
 |-- PeriodoDeAula: string (nullable = false)
 |-- FezAulaParticular: integer (nullable = false)
 |-- EstadoCivilDosPais: string (nullable = false)
 |-- PraticaEsportes: integer (nullable = false)
 |-- FilhoMaisVelho: string (nullable = false)
 |-- NumIrmaos: integer (nullable = false)
 |-- TransporteEscolar: integer (nullable = false)
 |-- HorasDeEstudo: double (nullable = true)
 |-- notaDeMatematica: integer (nullable = true)
 |-- notaDePortugues: integer (nullable = true)
 |-- notaDeCiencias: integer (nullable = true)



In [27]:


spark = SparkSession.builder \
    .appName("Analise de Renda e Notas") \
    .getOrCreate()


media_notas_por_renda = df3.groupBy("RendaFamiliar").agg(
    avg("NotaDeMatematica").alias("MediaMatematica"),
    avg("NotaDePortugues").alias("MediaPortugues"),
    avg("NotaDeCiencias").alias("MediaCiencias")
)


media_notas_por_renda.show()

corr_estudo_matematica = df3.stat.corr("HorasDeEstudo", "NotaDeMatematica")
corr_estudo_portugues = df3.stat.corr("HorasDeEstudo", "NotaDePortugues")
corr_estudo_ciencias = df3.stat.corr("HorasDeEstudo", "NotaDeCiencias")

df3 = df3.withColumn("FezAulaParticular", col("FezAulaParticular").cast("double"))  # Converter para numérico
corr_aula_particular_matematica = df3.stat.corr("FezAulaParticular", "NotaDeMatematica")
corr_aula_particular_portugues = df3.stat.corr("FezAulaParticular", "NotaDePortugues")
corr_aula_particular_ciencias = df3.stat.corr("FezAulaParticular", "NotaDeCiencias")

df3 = df3.withColumn("PraticaEsportes", col("PraticaEsportes").cast("double"))  # Converter para numérico
corr_esportes_matematica = df3.stat.corr("PraticaEsportes", "NotaDeMatematica")
corr_esportes_portugues = df3.stat.corr("PraticaEsportes", "NotaDePortugues")
corr_esportes_ciencias = df3.stat.corr("PraticaEsportes", "NotaDeCiencias")

print(f"Correlação entre Horas de Estudo e Notas:")
print(f"Matemática: {corr_estudo_matematica}")
print(f"Português: {corr_estudo_portugues}")
print(f"Ciências: {corr_estudo_ciencias}")

print(f"\nCorrelação entre Fez Aula Particular e Notas:")
print(f"Matemática: {corr_aula_particular_matematica}")
print(f"Português: {corr_aula_particular_portugues}")
print(f"Ciências: {corr_aula_particular_ciencias}")

print(f"\nCorrelação entre Pratica Esportes e Notas:")
print(f"Matemática: {corr_esportes_matematica}")
print(f"Português: {corr_esportes_portugues}")
print(f"Ciências: {corr_esportes_ciencias}")


+-------------+-----------------+-----------------+-----------------+
|RendaFamiliar|  MediaMatematica|   MediaPortugues|    MediaCiencias|
+-------------+-----------------+-----------------+-----------------+
| Desconhecida|66.18586956521739|68.91576086956522|67.90054347826087|
|      grupo A|62.99188823794502|66.78774222622803|65.25191527715187|
|      grupo D|67.66640010662401|70.38224710115954|70.89084366253499|
|      grupo C|64.69344333478072|68.43823273990448|66.99924012158054|
|      grupo E|75.29893590695373|74.25142291512002|72.67706013363029|
|      grupo B|63.49012875536481|67.32171673819742|65.89648068669528|
+-------------+-----------------+-----------------+-----------------+

Correlação entre Horas de Estudo e Notas:
Matemática: nan
Português: nan
Ciências: nan

Correlação entre Fez Aula Particular e Notas:
Matemática: nan
Português: nan
Ciências: nan

Correlação entre Pratica Esportes e Notas:
Matemática: nan
Português: nan
Ciências: nan
